# Responses: 2. Media

Here, [Attachment](../apiref/chatsky.script.core.message.rst#chatsky.script.core.message.Attachment) class is shown.
Attachments can be used for attaching different media elements
(such as [Image](../apiref/chatsky.script.core.message.rst#chatsky.script.core.message.Image),
[Document](../apiref/chatsky.script.core.message.rst#chatsky.script.core.message.Document)
or [Audio](../apiref/chatsky.script.core.message.rst#chatsky.script.core.message.Audio)).

They can be attached to any message but will only work if the chosen
[messenger interface](../apiref/index_messenger_interfaces.rst) supports them.

In [1]:
# installing dependencies
%pip install -q chatsky


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
from chatsky.script import RESPONSE, TRANSITIONS
from chatsky.script.conditions import std_conditions as cnd

from chatsky.script.core.message import Image, Message

from chatsky.pipeline import Pipeline
from chatsky.utils.testing import (
    check_happy_path,
    is_interactive_mode,
    run_interactive_mode,
)

In [3]:
img_url = "https://www.python.org/static/img/python-logo.png"
toy_script = {
    "root": {
        "start": {
            RESPONSE: Message(""),
            TRANSITIONS: {("pics", "ask_picture"): cnd.true()},
        },
        "fallback": {
            RESPONSE: Message(
                text="Final node reached, send any message to restart."
            ),
            TRANSITIONS: {("pics", "ask_picture"): cnd.true()},
        },
    },
    "pics": {
        "ask_picture": {
            RESPONSE: Message("Please, send me a picture url"),
            TRANSITIONS: {
                ("pics", "send_one", 1.1): cnd.regexp(r"^http.+\.png$"),
                ("pics", "send_many", 1.0): cnd.regexp(
                    f"{img_url} repeat 10 times"
                ),
                ("pics", "repeat", 0.9): cnd.true(),
            },
        },
        "send_one": {
            RESPONSE: Message(
                text="here's my picture!",
                attachments=[Image(source=img_url)],
            ),
            TRANSITIONS: {("root", "fallback"): cnd.true()},
        },
        "send_many": {
            RESPONSE: Message(
                text="Look at my pictures",
                attachments=[Image(source=img_url)],
            ),
            TRANSITIONS: {("root", "fallback"): cnd.true()},
        },
        "repeat": {
            RESPONSE: Message(
                text="I cannot find the picture. Please, try again."
            ),
            TRANSITIONS: {
                ("pics", "send_one", 1.1): cnd.regexp(r"^http.+\.png$"),
                ("pics", "send_many", 1.0): cnd.regexp(
                    r"^http.+\.png repeat 10 times"
                ),
                ("pics", "repeat", 0.9): cnd.true(),
            },
        },
    },
}

happy_path = (
    (Message("Hi"), Message("Please, send me a picture url")),
    (
        Message("no"),
        Message("I cannot find the picture. Please, try again."),
    ),
    (
        Message(img_url),
        Message(
            text="here's my picture!",
            attachments=[Image(source=img_url)],
        ),
    ),
    (
        Message("ok"),
        Message("Final node reached, send any message to restart."),
    ),
    (Message("ok"), Message("Please, send me a picture url")),
    (
        Message(f"{img_url} repeat 10 times"),
        Message(
            text="Look at my pictures",
            attachments=[Image(source=img_url)],
        ),
    ),
    (
        Message("ok"),
        Message("Final node reached, send any message to restart."),
    ),
)

In [4]:
pipeline = Pipeline.from_script(
    toy_script,
    start_label=("root", "start"),
    fallback_label=("root", "fallback"),
)

if __name__ == "__main__":
    check_happy_path(pipeline, happy_path)
    if is_interactive_mode():
        run_interactive_mode(pipeline)

(user) >>> text='Hi'
 (bot) <<< text='Please, send me a picture url'
(user) >>> text='no'
 (bot) <<< text='I cannot find the picture. Please, try again.'
(user) >>> text='https://www.python.org/static/img/python-logo.png'
 (bot) <<< text='here's my picture!' attachments='[{'chatsky_attachment_type': 'image', 'source': Url('https://www.python.org/static/img/python-logo.png'), 'use_cache': True}]'
(user) >>> text='ok'
 (bot) <<< text='Final node reached, send any message to restart.'
(user) >>> text='ok'
 (bot) <<< text='Please, send me a picture url'
(user) >>> text='https://www.python.org/static/img/python-logo.png repeat 10 times'
 (bot) <<< text='Look at my pictures' attachments='[{'chatsky_attachment_type': 'image', 'source': Url('https://www.python.org/static/img/python-logo.png'), 'use_cache': True}]'
(user) >>> text='ok'
 (bot) <<< text='Final node reached, send any message to restart.'
